In [175]:
import http.cookiejar
import json
import re
from http.cookiejar import MozillaCookieJar

import requests
from bs4 import BeautifulSoup

In [2]:
path = "../cookie.json"

In [3]:
def load_cookies_json(path):
    cookie_jar = MozillaCookieJar()

    with open(path) as f:
        file = json.loads(f.read())

    for cookie in file:

        if cookie.get("expirationDate") != None:
            exp = str(cookie.get("expirationDate", ""))

        elif cookie.get("expiry") != None:
            exp = str(cookie.get("expiry", ""))

        c = http.cookiejar.Cookie(
            version=0,
            name=cookie.get("name", ""),
            value=cookie.get("value", ""),
            port=None,
            port_specified=False,
            domain=cookie.get("domain", ""),
            domain_specified=True,
            domain_initial_dot=False,
            path=cookie.get("path", ""),
            path_specified=True,
            secure=False,
            expires=exp,
            discard=False,
            comment=None,
            comment_url=None,
            rest="",
        )

        cookie_jar.set_cookie(c)

    return cookie_jar

In [32]:
def searcher(link):
    header = {
        "Accept-Language": "en-US,en;q=0.5",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0",
    }
    session.headers = header
    data = session.get(link)
    soup = BeautifulSoup(data.content, "html.parser")
    return soup

# login Facebook

In [5]:
session = requests.Session()
session.cookies = load_cookies_json(path)

# user_id , Name

In [12]:
response = searcher("https://www.facebook.com/profile.php")
data = {}

t = re.compile(r"USER_ID.+?SHORT_NAME").search(str(response)).group(0)
USER_ID, NAME = t.replace('"', "").replace(",SHORT_NAME", "").split(",")
USER_ID = USER_ID.split(":")[-1]
NAME = NAME.split(":")[-1]

data["username"] = (
    re.search("userVanity.+?,", str(response))
    .group(0)
    .replace('"', "")
    .replace(",", "")
    .split(":")[-1]
)
data["uid"] = USER_ID
data["name"] = NAME
print(data)

{'username': 'SWEETERROR404', 'uid': '100004486669038', 'name': 'Mubeen Ahmad'}


# Extract accounts

In [227]:
link = ["https://mbasic.facebook.com/friends/center/requests/outgoing/"]
uids = []
for i in link:
    try:
        res = searcher(i)
        see_more = 'https://mbasic.facebook.com'+re.search('href="/friends/center/requests/outgoing/.+?"',str(res)).group(0).replace('"','').replace('&amp;','').replace('href=','')
        uid = re.findall('uid=(.+?)&amp;',str(res))
        if len(uid) == 0:
            break
        uids.extend(uid)
        link.append(see_more)
        
    except:
        print("Link Complete")


In [228]:
print('total accounts',len(uids))

total accounts 345


# create params

In [229]:
import concurrent.futures

params = {}

def uids_params(uid):
    res = searcher("https://mbasic.facebook.com/"+uid)
    pr = dict(i.split('=') for i in re.search('subject_id.+?"',str(res)).group(0).replace('"','').replace('cancel/?','').split('&amp;'))
    params[pr['subject_id']] = pr

def process_links(links):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(uids_params, links)

process_links(uids)

In [230]:
len(params)

344

# cancel all send requests

In [234]:
def cancel_request(p):
    session.get('https://mbasic.facebook.com/a/friendrequest/cancel/', 
                params=p)
    
def process_links(links):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(cancel_request, links)

process_links(list(params.values()))